# Finding alpha signals in insider trading

In this post, we’ll take a look at insider trading (the legal kind, not the illegal kind) and what information, if any, investors can draw from the required disclosures that insiders must make.

In [1]:
import pandas as pd
from fidap import fidap_client

## What is insider trading

Insider trading generally has a negative connotation of being illegal. However, that’s most often not the case - the vast majority of insider trading is perfectly legal. An “insider” is any individual that may have material nonpublic information about a company. Generally, the definition includes the following people - 

1. Company directors
2. Company officers
3. Any shareholder with >10% holdings
4. Often, corporate insiders will need to trade in the shares of the company. Whenever an insider transacts in the shares of the company, they must file a declaration with the SEC called a Form 4. A filing looks something like this - 

More detailed information about what these fields mean can be found here.


## Is there information in insider trading?

For investors, the key question is whether or not insider trades can be used as a signal. There are two key questions we need to ask ourselves - 

1. What is the reason behind the insider trade?
2. If the reason behind the insider trade is price-related (eg an opinion on whether the price will go up or down), are insiders accurate?

There could be a variety of reasons that insiders trade in a security beyond whether they think the price is going to go up or down. For company directors and officers, a large part of their compensation may be based in shares, and they may decide to liquidate those shares for diversification or for liquidity needs for other reasons, such as taxes or other purchases. Though less common, even insider buys may have motivations driven by requirements, such as the need to maintain a certain level of ownership.

However, if the motivation of the insider trade is indeed an opinion on the future trajectory of the price of the stock, then we may conclude that insiders have additional information about the future prospects of the company. 

Therefore, can we assume that insiders buying shares is a positive signal and insiders selling shares is a negative signal?



## Data exploration using Fidap

Let’s see what the data tells us. Fidap has a dataset around insider trades that covers all SEC filings. Here’s a link to the table details page. There are 24 columns and 11.9m rows. The columns include information on the date of the transaction and the date of the filing, the transaction amount, the owner, the underlying security, as well as several other details.

The columns are below - 



In [2]:
fidap = fidap_client(db="sf", api_key="1287a2d9e40042edb53254cd203b9a0b")

In [3]:
fidap.sql("""
    select * from meta_fields where "table" = 'insider_trades'
""")

,table,NAME,DISPLAY_NAME,DESCRIPTION,PD_TYPE,PCT_FILLED,UNIQUENESS,MEAN,MEDIAN
0,insider_trades,ticker,Ticker Symbol,The ticker is a unique identifer for an issuer...,object,1.000000,1.516150e-03,NaN,NaN
1,insider_trades,filingdate,Filing Date,The date the form was filed with the SEC.,datetime64[ns],1.000000,3.774522e-04,NaN,NaN
2,insider_trades,formtype,Form Type,"""The type of SEC form . Available options are ...",object,1.000000,6.917000e-07,NaN,NaN
3,insider_trades,issuername,Issuer Name,The name of the security issuer.,object,0.990252,1.494591e-03,NaN,NaN
4,insider_trades,ownername,Owner Name (Insider / Investor),The name of the owner.,object,1.000000,1.886212e-02,NaN,NaN
5,insider_trades,officertitle,Officer Title,Is the owner is an officer of the company the ...,object,0.606570,7.799448e-03,NaN,NaN
6,insider_trades,isdirector,Is Director?,Is the owner a Board Director? [Y]es or [N]o.,object,1.000000,2.306000e-07,NaN,NaN
7,insider_trades,isofficer,Is Officer?,Is the owner an officer of the company? [Y]es ...,object,1.000000,2.306000e-07,NaN,NaN
8,insider_trades,istenpercentowner,Is Ten Percent Owner?,Does the owner hold ten percent or more of the...,object,1.000000,2.306000e-07,NaN,NaN
9,insider_trades,transactiondate,Transaction Date,If there has been a transaction; the date of t...,datetime64[ns],0.688095,7.890295e-04,NaN,NaN


Over fifteen years that we have data in Fidap, we see a total 12 million transactions, covering 15,000 tickers, and a whopping $9 trillion in transaction volume. See the query below

In [4]:
fidap.sql("""
    select count(*) as count, sum(transactionvalue) as tvalue, 
    count(distinct(ticker)) as tickers, min(filingdate) from insider_trades
""")

,COUNT,TVALUE,TICKERS,MIN(FILINGDATE)
0,7223907,6669452022398,13120,2008-01-02


Let's first explore the data and find some basic numbers here.

We can see these same numbers in the Fidap dashboard as well.

Let's see some of the individual rows here.

In [5]:
fidap.sql("""
    select * from insider_trades where ticker='STT'  order by filingdate desc limit 5
""")

,TICKER,FILINGDATE,FORMTYPE,ISSUERNAME,OWNERNAME,OFFICERTITLE,ISDIRECTOR,ISOFFICER,ISTENPERCENTOWNER,TRANSACTIONDATE,SECURITYADCODE,TRANSACTIONCODE,SHARESOWNEDBEFORETRANSACTION,TRANSACTIONSHARES,SHARESOWNEDFOLLOWINGTRANSACTION,TRANSACTIONPRICEPERSHARE,TRANSACTIONVALUE,SECURITYTITLE,DIRECTORINDIRECT,NATUREOFOWNERSHIP,DATEEXERCISABLE,PRICEEXERCISABLE,EXPIRATIONDATE,ROWNUM
0,STT,2020-11-20,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,2020-11-19,ND,S,18704,-400.0,18304,68.59,27436.0,Common Stock,D,None,NaN,NaN,NaN,1
1,STT,2020-11-20,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,None,N,None,550,NaN,550,NaN,NaN,Common Stock,I,By domestic partner,NaN,NaN,NaN,2
2,STT,2020-11-17,4,STATE STREET CORP,PHELAN DAVID C,EVP; Gen Counsel and Secretary,N,Y,N,2020-11-15,ND,F,98071,-1822.0,96249,68.52,124843.0,Common Stock,D,None,NaN,NaN,NaN,1
3,STT,2020-11-17,4,STATE STREET CORP,TARAPOREVALA CYRUS,EVP; President and CEO of SSGA,N,Y,N,2020-11-15,ND,F,66730,-566.0,66164,68.52,38782.0,Common Stock,D,None,NaN,NaN,NaN,1
4,STT,2020-11-17,4,STATE STREET CORP,RICHARDS MICHAEL L,EVP and Chief Admin Officer,N,Y,N,2020-11-15,ND,F,19801,-781.0,19020,68.52,53514.0,Common Stock,D,None,NaN,NaN,NaN,1


We can actually get info on each of the columns from the metadata table. Let's take a look.

In [6]:
fidap.sql("""
    select * from insider_trades where ticker='AAPL' and securitytitle = 'Common Stock' limit 5
""")

,TICKER,FILINGDATE,FORMTYPE,ISSUERNAME,OWNERNAME,OFFICERTITLE,ISDIRECTOR,ISOFFICER,ISTENPERCENTOWNER,TRANSACTIONDATE,SECURITYADCODE,TRANSACTIONCODE,SHARESOWNEDBEFORETRANSACTION,TRANSACTIONSHARES,SHARESOWNEDFOLLOWINGTRANSACTION,TRANSACTIONPRICEPERSHARE,TRANSACTIONVALUE,SECURITYTITLE,DIRECTORINDIRECT,NATUREOFOWNERSHIP,DATEEXERCISABLE,PRICEEXERCISABLE,EXPIRATIONDATE,ROWNUM
0,AAPL,2018-02-20,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2018-02-15,None,M,8619,675,9294,NaN,NaN,Common Stock,D,NaN,NaN,NaN,NaN,1
1,AAPL,2018-02-20,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2018-02-15,ND,F,9294,-229,9065,172.99,39615.0,Common Stock,D,NaN,NaN,NaN,NaN,2
2,AAPL,2016-02-17,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2016-02-15,None,M,12651,675,13326,NaN,NaN,Common Stock,D,NaN,NaN,NaN,NaN,1
3,AAPL,2016-02-17,4,APPLE INC,KONDO CHRIS,Principal Accounting Officer,N,Y,N,2016-02-15,ND,F,13326,-276,13050,93.99,25941.0,Common Stock,D,NaN,NaN,NaN,NaN,2
4,AAPL,2015-02-20,4,APPLE INC,JUNG ANDREA,None,Y,N,N,2015-02-18,None,M,14595,40000,54595,25.72,1028800.0,Common Stock,D,NaN,NaN,NaN,NaN,1


In [7]:
# need to link with tickers table
fidap.sql("""
    select ticker, count(*) as count, sum(transactionvalue) as tvalue from insider_trades
    where transactionvalue > 0 group by ticker order by tvalue desc limit 10
""")

,TICKER,COUNT,TVALUE
0,NGBL,995,1425086696067
1,GGP,497,73606075821
2,PWRM,75,61806147612
3,TELOZ,63,50915143904
4,HLT,363,50495362012
5,AMGTI,348,49325889352
6,INVH,347,41747721180
7,OMAG,51,41030154474
8,CHTRQ,864,40386743764
9,FB,6877,40168004044


In [8]:
fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionvalue from insider_trades, tickers
    where insider_trades.ticker = tickers.ticker and insider_trades.transactionvalue > 100000 and insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
    limit 100
""")

,TICKER,FILINGDATE,TRANSACTIONVALUE
0,TDC,2017-02-24,242314
1,CSCO,2017-02-24,472434
2,CSCO,2017-02-24,8515000
3,CSCO,2017-02-24,775102
4,CSCO,2017-02-24,775330
...,...,...,...
95,CRM,2019-02-15,857751
96,ACLS,2019-02-15,307757
97,VCRA,2019-02-15,151550
98,CVLT,2019-02-15,1449533


In [10]:
df = fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionshares,
    insider_trades.transactionvalue, daily.close, daily.fc_1m
    from insider_trades, tickers, daily
    where insider_trades.ticker = 'AAPL' and
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
""")

In [11]:
df

,TICKER,FILINGDATE,TRANSACTIONSHARES,TRANSACTIONVALUE,CLOSE,FC_1M
0,AAPL,2016-02-03,-2008,191704,24.087,0.057334
1,AAPL,2016-03-08,-15088,1554215,25.258,0.075501
2,AAPL,2016-03-17,-18241,1895605,26.450,0.010473
3,AAPL,2016-03-23,-256356,27150664,26.532,-0.009875
4,AAPL,2016-03-23,-256356,27150664,26.532,-0.009875
...,...,...,...,...,...,...
357,AAPL,2020-10-19,-12452,1495983,115.980,0.017675
358,AAPL,2020-11-05,-2612,285283,119.030,0.044947
359,AAPL,2020-11-05,-8641,953966,119.030,0.044947
360,AAPL,2020-11-05,-5747,637860,119.030,0.044947


In [12]:
df['FC_1M'].mean()

0.030035378544751384

In [17]:
df[df['TRANSACTIONSHARES'] > 0]['FC_1M'].mean()

0.024119978448000002

In [13]:
df[df['TRANSACTIONSHARES'] < 0]['FC_1M'].mean()

0.029390211910115607

In [14]:
len(df[df['TRANSACTIONSHARES'] > 0]['FC_1M'])

16

In [15]:
df2 = fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionshares,
    insider_trades.transactionvalue, daily.close, daily.fc_1m
    from insider_trades, tickers, daily
    where 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
""")

In [16]:
df2[df2['TRANSACTIONSHARES'] > 0]['FC_1M'].mean()

0.01672661702722382

In [17]:
df2[df2['TRANSACTIONSHARES'] < 0]['FC_1M'].mean()

0.01957024488006686

In [18]:
fidap.sql("""
    select avg(daily.fc_3m), avg(daily.fc_1m), count(daily.fc_3m)
    from insider_trades, tickers, daily
    where 
    insider_trades.transactionshares > 0 and 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock'
""")

,AVG(DAILY.FC_3M),AVG(DAILY.FC_1M),COUNT(DAILY.FC_3M)
0,0.060503,0.017141,70341


In [19]:
fidap.sql("""
    select avg(daily.fc_3m), avg(daily.fc_1m), count(daily.fc_3m)
    from insider_trades, tickers, daily
    where 
    insider_trades.transactionshares < 0 and 
    insider_trades.ticker = tickers.ticker and 
    insider_trades.ticker = daily.ticker and
    insider_trades.filingdate = daily.date and
    insider_trades.transactionvalue > 100000 and 
    insider_trades.securitytitle = 'Common Stock' 
""")

,AVG(DAILY.FC_3M),AVG(DAILY.FC_1M),COUNT(DAILY.FC_3M)
0,0.056397,0.00895,174460


In [20]:
fidap.sql("""select * from fundamentals where ticker='URBN' and dimension = 'MRQ' order by calendardate desc""")

,TICKER,DIMENSION,CALENDARDATE,DATEKEY,REPORTPERIOD,LASTUPDATED,ACCOCI,ASSETS,ASSETSAVG,ASSETSC,ASSETSNC,ASSETTURNOVER,BVPS,CAPEX,CASHNEQ,CASHNEQUSD,COR,CONSOLINC,CURRENTRATIO,DE,DEBT,DEBTC,DEBTNC,DEBTUSD,DEFERREDREV,DEPAMOR,DEPOSITS,DIVYIELD,DPS,EBIT,EBITDA,EBITDAMARGIN,EBITDAUSD,EBITUSD,EBT,EPS,EPSDIL,EPSUSD,EQUITY,EQUITYAVG,EQUITYUSD,EV,EVEBIT,EVEBITDA,FCF,FCFPS,FXUSD,GP,GROSSMARGIN,INTANGIBLES,INTEXP,INVCAP,INVCAPAVG,INVENTORY,INVESTMENTS,INVESTMENTSC,INVESTMENTSNC,LIABILITIES,LIABILITIESC,LIABILITIESNC,MARKETCAP,NCF,NCFBUS,NCFCOMMON,NCFDEBT,NCFDIV,NCFF,NCFI,NCFINV,NCFO,NCFX,NETINC,NETINCCMN,NETINCCMNUSD,NETINCDIS,NETINCNCI,NETMARGIN,OPEX,OPINC,PAYABLES,PAYOUTRATIO,PB,PE,PE1,PPNENET,PREFDIVIS,PRICE,PS,PS1,RECEIVABLES,RETEARN,REVENUE,REVENUEUSD,RND,ROA,ROE,ROIC,ROS,SBCOMP,SGNA,SHAREFACTOR,SHARESBAS,SHARESWA,SHARESWADIL,SPS,TANGIBLES,TAXASSETS,TAXEXP,TAXLIABILITIES,TBVPS,WORKINGCAPITAL,REV_YOY,REV_QOQ
0,URBN,MRQ,2020-09-30,2020-10-31,2020-10-31,2020-12-11,-30894000,3530675000,NaN,1371561000,2159114000,NaN,14.637,-17050000,624945000,624945000,646666000,76704000,1.448,1.467,1324556000,255122000,1069434000,1324556000,0,74463000,0,0,0,97618000,172081000,0.177,172081000,97618000,97618000,0.78,0.78,0.78,1431319000,NaN,1431319000,3014263752,399,9.719,82403000,0.843,1,322941000,0.333,0,0,3283388000,NaN,489234000,9352000,2000,9350000,2099356000,946898000,1152458000,2184547752,-37915000,0,-60000,-120000000,0,-120060000,-16103000,947000,99453000,-1205000,76704000,76704000,76704000,0,0,0.079,224433000,98508000,349793000,0,1.526,-280.106,-279.250,2032059000,0,22.34,0.619,0.619,87187000,1446534000,969607000,969607000,0,NaN,NaN,NaN,NaN,5773000,224433000,1,97786381,97784661,98583032,9.916,3530675000,117705000,20914000,0,36.107,424663000,-0.018089,0.207081
1,URBN,MRQ,2020-06-30,2020-07-31,2020-07-31,2020-12-11,-29203000,3425278000,NaN,1270966000,2154312000,NaN,13.813,-28585000,662860000,662860000,565228000,34400000,1.648,1.536,1492576000,270326000,1222250000,1492576000,0,74749000,0,0,0,68886000,143635000,0.179,143635000,68886000,68886000,0.35,0.35,0.35,1350593000,NaN,1350593000,2581137352,-181,8.896,146283000,1.496,1,238038000,0.296,0,0,3483778000,NaN,351771000,9717000,501000,9216000,2074685000,771216000,1303469000,1617274352,74120000,0,-22000,-100000000,0,-100022000,-4219000,24366000,174868000,3493000,34400000,34400000,34400000,0,0,0.043,168619000,69419000,207261000,0,1.197,-56.056,-57.034,2023804000,0,16.54,0.456,0.456,60441000,1369830000,803266000,803266000,0,NaN,NaN,NaN,NaN,6385000,168619000,1,97779586,97778749,98104918,8.215,3425278000,121292000,34486000,0,35.031,499750000,-0.165290,0.364977
2,URBN,MRQ,2020-03-31,2020-04-30,2020-04-30,2020-12-11,-40925000,3356204000,NaN,1176928000,2179276000,NaN,13.258,-43518000,588740000,588740000,576640000,-138442000,1.774,1.585,1552603000,243671000,1308932000,1552603000,0,76294000,0,0,0,-198573000,-122279000,-0.208,-122279000,-198573000,-198573000,-1.41,-1.41,-1.41,1298108000,NaN,1298108000,2832707763,-1824,9.342,-103170000,-1.054,1,11843000,0.020,0,0,3656490000,NaN,335640000,78393000,65121000,13272000,2058096000,663577000,1394519000,1695458763,366901000,0,-10756000,220000000,0,209244000,222223000,265741000,-59652000,-4914000,-138442000,-138442000,-138442000,0,0,-0.235,210578000,-198735000,104702000,0,1.306,-578.457,-578.000,1996950000,0,17.34,0.457,0.458,55910000,1335430000,588483000,588483000,0,NaN,NaN,NaN,NaN,4872000,210578000,1,97777322,97910314,97910314,6.010,3356204000,169054000,-60131000,0,34.278,513351000,-0.319211,-0.496842
3,URBN,MRQ,2019-12-31,2020-01-31,2020-01-31,2020-12-11,-28004000,3315633000,NaN,1053396000,2262237000,NaN,14.857,-46312000,221839000,221839000,835785000,19539000,1.649,1.278,1359088000,221593000,1137495000,1359088000,0,77081000,0,0,0,39616000,116697000,0.100,116697000,39616000,39616000,0.20,0.20,0.20,1455355000,NaN,1455355000,3674349781,15,6.771,106179000,1.084,1,333793000,0.285,0,0,3814112000,NaN,409534000,308549000,211453000,97096000,1860278000,638770000,1221508000,2508

In [22]:
fidap.sql("""select * from meta_fields where "table" = 'fundamentals'""")

,table,NAME,DISPLAY_NAME,DESCRIPTION,PD_TYPE,PCT_FILLED,UNIQUENESS,MEAN,MEDIAN
0,fundamentals,ticker,Ticker Symbol,[Entity] The ticker is a unique identifer for ...,object,1.000000,0.016756,NaN,NaN
1,fundamentals,dimension,Dimension,[Entity] The dimension field allows you to tak...,object,1.000000,0.000014,NaN,NaN
2,fundamentals,calendardate,Calendar Date,"""[Entity] The Calendar Date represents the nor...",datetime64[ns],1.000000,0.000046,NaN,NaN
3,fundamentals,datekey,Date Key,[Entity] The Date Key represents the SEC filin...,datetime64[ns],1.000000,0.003615,NaN,NaN
4,fundamentals,reportperiod,Report Period,[Entity] The Report Period represents the end ...,datetime64[ns],1.000000,0.001766,NaN,NaN
...,...,...,...,...,...,...,...,...,...
108,fundamentals,taxliabilities,Tax Liabilities,[Balance Sheet] A component of [Liabilities] r...,float64,0.987963,0.074720,2.241745e+09,0.000000e+00
109,fundamentals,tbvps,Tangible Assets Book Value per Share,[Metrics] Measures the ratio between [Tangible...,float64,0.984843,0.159967,1.501994e+04,1.797900e+01
110,fundamentals,workingcapital,Working Capital,[Metrics] Working capital measures the differe...,float64,0.781734,0.166667,2.253482e+09,5.306800e+07
111,fundamentals,rev_yoy,None,None,float64,0.150441,0.145233,NaN,5.497050e-02
